In [8]:
from scipy.spatial.transform import Rotation as R
import numpy as np
import numpy.typing as npt
import util

# Rotations (Euler and Rotatation vectors)

In [9]:
# from euler, extrinsic rotation
## single axis
R.from_euler(seq='x', angles=90, degrees=True).as_matrix()   
R.from_euler('x', [90], degrees=True).as_matrix()  
R.from_euler('x', [90, 45, 30], degrees=True).as_matrix() 

## multi axis
R.from_euler('xyz', [90, 45, 30], degrees=True).as_matrix()
R.from_euler('xyz', [[90, 45, 30]], degrees=True).as_matrix()
R.from_euler('xyz', [[90, 45, 30], [35, 45, 90]], degrees=True).as_matrix()

array([[[ 6.12372436e-01,  6.12372436e-01,  5.00000000e-01],
        [ 3.53553391e-01,  3.53553391e-01, -8.66025404e-01],
        [-7.07106781e-01,  7.07106781e-01,  5.55111512e-17]],

       [[ 1.66533454e-16, -8.19152044e-01,  5.73576436e-01],
        [ 7.07106781e-01,  4.05579788e-01,  5.79227965e-01],
        [-7.07106781e-01,  4.05579788e-01,  5.79227965e-01]]])

Assuming universal robot joints are in Base: 0, -91.66, -68.04, -110.30, 90, Wrist 3: 0 </br>
we can use `var:=get_actual_tcp_pose()`  to get the actual pose of tcp wrt to base </br>
we'll get p[0.47987, -0.1333, 0.6238, 2.22144, -2.22144, -0] </br>

<p align="center">
  <img src=".\imgs\2024-12-21 204339.png" alt="" width="400" style="max-width:100%;">
</p>

code:
```python
x = R.from_rotvec([2.22144, -2.22144, 0]).as_matrix()
util.print_array(x, True, 3)  

"""
[[ 0. -1. -0.]
 [-1.  0. -0.]
 [ 0.  0. -1.]]

1st col: indicates x axis of tcp is in -1 dir of y axis of base
2nd col: indicates y axis of tcp is in -1 dir of x axis of base
3rd col: z axis of tcp is in -1 dir of z axis of base
"""

```
In the following, we can see RPY is [180.  -0. -90.]. The code below confirms that we need to use `"xyz"` seq in `as_euler`
<p align="center">
  <img src=".\imgs\2024-12-21 205258.png" alt="" width="400" style="max-width:100%;">
</p>

code:
```python
x = R.from_rotvec([2.22144, -2.22144, 0]).as_euler("xyz", degrees=True)
util.print_array(x, True, 3) # [180.  -0. -90.]
```


In [10]:
x = R.from_rotvec([2.22144, -2.22144, 0]).as_euler("xyz", degrees=True)
util.print_array(x, True, 3)  

[180.  -0. -90.]


# Universal robot 

## Calculating pose transformation with python vs using PolyScope 
- refer to Obsidian Notes for full setup of Polyscope. The code below will still work

In [11]:
"""
According to Copilot, we can also use cv2.Rodrigues

def pose_to_homogeneous(pose):
    x, y, z, rx, ry, rz = pose
    R, _ = cv2.Rodrigues(np.array([rx, ry, rz]))
    T = np.eye(4)
    T[:3, :3] = R
    T[:3, 3] = [x, y, z]
    return T

def homogeneous_to_pose(T):
    R = T[:3, :3]
    t = T[:3, 3]
    rx, ry, rz = cv2.Rodrigues(R)[0].flatten()
    return [t[0], t[1], t[2], rx, ry, rz]
"""

def pose_to_TMatrix(pose: list[float]|npt.ArrayLike):
    x, y, z, rx, ry, rz = pose
    rot = R.from_rotvec([rx, ry, rz])
    m = np.eye(4)
    m[:3, :3] = rot.as_matrix()
    m[:3, 3] = [x, y, z]
    return m

def TMatrix_to_pose(m: npt.NDArray):
    rot = R.from_matrix(m[:3, :3])
    t = m[:3, 3]
    rx, ry, rz = rot.as_rotvec()
    return [t[0], t[1], t[2], rx, ry, rz]

def inverse_TMatrix(T):
    R = T[:3, :3]
    t = T[:3, 3]
    R_inv = R.T
    t_inv = -np.dot(R_inv, t)
    T_inv = np.eye(4)
    T_inv[:3, :3] = R_inv
    T_inv[:3, 3] = t_inv
    return T_inv

# starting tcp pose relative to base frame (movej, variable position)
pose_A = [0.03979, -0.44943, 0.48438, -0.00122, 3.14159, 0]

# delta pose relative to A
delta_ab = [0.18375, 0.01395, 0.23235, 0, 0, 0]
delta_ac = [0, 0, 0.35, 0,0,0]

"""
Typically, without knowing the transofrmation matrix from B to C, 
all we can do is:

movej(pose_A)
movel(pose_B)
movej(pose_A)
movel(pose_C)

To move directly from B to C (there by skipping the 2nd movej(A)), we need
to perform the following calculation:

T_{AC} = T_{AB}\;T_{BC} \\
T^{-1}_{AB}\;T_{AC} = T_{BC}

Steps:
1. Convert the pose to a homogeneous transformation matrix.
2. Calculate the inverse of T_{AB}.
3. Multiply the inverse of T_{AB} with T_{AC}.
4. For use with cobot on PolyScope, convert the resulting matrix back to a pose.
5. Use moveL with this pose.
"""

t_AB = pose_to_TMatrix(delta_ab)
t_AC = pose_to_TMatrix(delta_ac)
t_ab_inv = inverse_TMatrix(t_AB) # pose_inv(t_AB) in Polyscope
t_bc = t_ab_inv @ t_AC # pose_trans(t_ab_inv, t_AC) in Polyscope

delta_bc = TMatrix_to_pose(t_bc)
print("delta_bc")
util.print_array(np.array(delta_bc), True, precision=7)
# [-0.18375 -0.01395  0.11765  0.       0.       0.     ]

delta_bc
[-0.18375 -0.01395  0.11765  0.       0.       0.     ]
